<a href="https://colab.research.google.com/github/skurakake/MMP_public/blob/main/Resnet34_C100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
MODEL_DIR = '/content/drive/MyDrive/Colab Notebooks/temp'

if not os.path.exists(MODEL_DIR):  # ディレクトリが存在しない場合、作成する。
    os.makedirs(MODEL_DIR)


In [ ]:
# cifar100 を使う
from tensorflow.keras.datasets import cifar100

(x_train, t_train), (x_test, t_test) = cifar100.load_data(label_mode='fine')


CIFAR100_LABELS_LIST = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle',
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel',
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock',
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster',
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

num_category = 100 # クラス数

In [ ]:
# ダウンロードしたデータの一部をランダムに表示

print('x_train shape:', x_train.shape)
print('the number of train samples:', x_train.shape[0] )
print('the number of test samples:', x_test.shape[0] )
print(type(x_train[0]))

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)
print(type(x_test))
print(type(t_test[0]))


num_image = 20 # 表示数
rand_idx = np.random.randint(0, len(x_train), num_image)
plt.figure(figsize=(5, 5)) # 表示領域のサイズ
# plt figure set to 5inch x 5inch(500pixel x 500 pixel).

labels = CIFAR100_LABELS_LIST
for i in range(num_image):
    plt.subplot(4, 5, i + 1)
    plt.imshow(x_train[rand_idx[i]])
    plt.title(labels[t_train[rand_idx[i]][0]])
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)

plt.show() # 実行されたとき始めてウィンドウが立ち上がり、そのウィンドウに図が表示される
# colab では plt.show() がなくても、環境が自動でウィンドウをオープンしてくれるが、
# コマンドラインとかでは、これが必要

In [ ]:
print("Number of training data", len(x_train))
print("Number of test data", len(x_test))
print("Number of label data for training", len(t_train))
print("Number of label data for test", len(t_test))

In [ ]:
x_train_N =x_train /255
x_test_N = x_test /255

In [ ]:
# 設定
batch_size = 32 # ミニバッチサイズ
epochs = 100 # エポック数
num_class = 100 # クラス数

# 正解データを one-hot 表現へ変換
t_train_n = tf.keras.utils.to_categorical(t_train, num_class)
t_test_n = tf.keras.utils.to_categorical(t_test, num_class)
print("正解データ（学習）：", t_train_n[5])
print("正解データ（テスト）：", t_test_n[22])

モデルの構築
Functional API を使う。
model = Model(inputs=input_tensor, outputs=outputs)　でモデルを定義する。

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAvgPool2D, Concatenate
from tensorflow.keras.layers import BatchNormalization, Resizing, Rescaling
from tensorflow.keras.optimizers import Adam

#  using Functional API
def Residual_block1(input_tensor, c1, c2):
    # Path 1 is a single 1 x 1 convolutional layer
    p1_1 = Conv2D(c1[0], c1[1], padding='same')(input_tensor)
    p1_2 = BatchNormalization()(p1_1)
    p1_3 = Activation('relu')(p1_2)
    p1_4 = Conv2D(c2[0], c2[1], padding='same')(p1_3)
    p1_5 = BatchNormalization()(p1_4)
    p1_6 = Conv2D(64, 1, padding='same')(p1_5)
    p1_7 = p1_6 + input_tensor
    p1_8 = Activation('relu')(p1_7)

    return p1_8


# Define the blocks using the Functional API
def b1(input_tensor):
    x = Resizing(224,224)(input_tensor)
    x = Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)

    return x

def b2(input_tensor):
    x = MaxPooling2D(pool_size=3, strides=2, padding='same')(input_tensor)
    x = Residual_block1(x, (64,3),(64,3))
    x = Residual_block1(x, (64,3),(64,3))
    x = Residual_block1(x, (64,3),(64,3))
    return x

def b3(input_tensor):
    x = Residual_block1(input_tensor, (128,3), (128,3))
    x = Residual_block1(x, (128,3), (128,3))
    x = Residual_block1(x, (128,3), (128,3))
    x = Residual_block1(x, (128,3), (128,3))
    return x

def b4(input_tensor):
    x = Residual_block1(input_tensor, (256, 3),(256, 3))
    x = Residual_block1(x, (256, 3),(256, 3))
    x = Residual_block1(x, (256, 3),(256, 3))
    x = Residual_block1(x, (256, 3),(256, 3))
    x = Residual_block1(x, (256, 3),(256, 3))
    x = Residual_block1(x, (256, 3),(256, 3))

    return x

def b5(input_tensor):
    x = Residual_block1(input_tensor, (512, 3),(512, 3))
    x = Residual_block1(x, (512, 3),(512, 3))
    x = Residual_block1(x, (512, 3),(512, 3))
    return x

# Build the complete model
input_tensor = Input(shape=x_train.shape[1:])
x = b1(input_tensor)
x = b2(x)
x = b3(x)
x = b4(x)
x = b5(x)
x = GlobalAvgPool2D()(x)
x = Flatten()(x)
x =Dropout(0.5)(x)
outputs = Dense(1000)(x)
x =Dropout(0.5)(x)
outputs = Dense(num_class, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=outputs)

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

In [ ]:
# 保存先のパスを設定。Googleドライブ内のパスを指定。
# checkpoint_filepath = os.path.join(MODEL_DIR, '/my_best_model.keras')
checkpoint_filepath = os.path.join(MODEL_DIR, 'my_Resnet1.keras')

# ModelCheckpoint コールバックを定義します
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',       # 監視する指標 (例: 検証精度)
    save_best_only=True,         # 最も性能の良いモデルのみを保存する
    save_weights_only=False,     # モデルのアーキテクチャとトレーニング設定も保存する
    mode='max',                  # 監視指標の最大値が最良であることを指定 ('max' or 'min')
    verbose=1                    # 保存時にメッセージを表示
)

In [ ]:
# モデルの読み込み
model.load_weights(os.path.join(MODEL_DIR, "my_Resnet1.keras"))  # のモデルを指定


モデルを訓練する

In [ ]:
#x_train_N =x_train /255
#x_test_N = x_test /255

history = model.fit(x_train, t_train_n, batch_size=batch_size,
                    epochs=epochs, validation_data=(x_test, t_test_n),
                    callbacks=[model_checkpoint_callback] # ここでコールバックを指定
                    )

In [ ]:
# 学習の経過をグラフで表示
import matplotlib.pyplot as plt
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
validation_loss = history.history['val_loss']
validation_accuracy = history.history['val_accuracy']

plt.plot(np.arange(len(train_loss)), train_loss, label='training_loss')
plt.plot(np.arange(len(validation_loss)), validation_loss, label='validation_loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

plt.plot(np.arange(len(train_accuracy)), train_accuracy, label='training_accuracy')
plt.plot(np.arange(len(validation_accuracy)), validation_accuracy, label='validation_accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()